In [29]:
import pandas as pd
import os
import numpy as np
from scipy.stats import mode

In [52]:
if os.path.exists('../data/df_7_34567.txt'):
    print('>>> read df_7_34567')
    df = pd.read_csv('../data/df_7_34567.txt', delimiter=';', parse_dates=['time_interval_begin'], dtype={'link_ID': object})
else:
    print('>>> save and read df_7_34567')
    df1 = pd.read_csv('../data/traveltime_2016_7_2017_4567.txt', delimiter=';', dtype={'link_ID': object})
    df2 = pd.read_csv('../data/traveltime_2017_3456.txt', delimiter=';', dtype={'link_ID': object})

    df2 = df2.rename(columns={'linkID': 'link_ID'})
    df1['time_interval_begin'] = pd.to_datetime(df1['time_interval'].map(lambda x: x[1:20]))
    df2['time_interval_begin'] = pd.to_datetime(df2['time_interval'].map(lambda x: x[1:20]))
    df2 = df2.loc[(df2['time_interval_begin'] >= pd.to_datetime('2017-03-01')) & 
                  (df2['time_interval_begin'] <= pd.to_datetime('2017-03-31'))]
    df = pd.concat([df1, df2])
    df = df.drop(['time_interval'], axis=1)
    df.to_csv('../data/df_7_34567.txt', header=True, index=None, sep=';', mode='w')

>>> read df_7_34567


In [53]:
df['time_interval_begin_minutes'] = df.time_interval_begin.dt.minute
# 四月份数据
train_data = df.loc[(df.time_interval_begin.dt.month == 4) & 
                    (df.time_interval_begin.dt.hour.isin([7, 8, 14, 15, 17, 18]))
                   ]
train_history = df.loc[(df.time_interval_begin.dt.month == 3)]

In [54]:
# mode函数：出现次数最多的数，相同次数则返回最小
def mode_function(df):
    counts = mode(df)
    return counts[0][0]

train_history = df.loc[(df.time_interval_begin.dt.month == 3)]
train_history = train_history.groupby(
    ['link_ID', 'time_interval_begin_minutes'])['travel_time'].agg(
    [('mean_m', np.mean), ('median_m', np.median), 
     ('mode_m', mode_function), ('std_m', np.std)
    ]).reset_index()
train_data = pd.merge(train_data, train_history, 
                      on=['link_ID', 'time_interval_begin_minutes'], 
                      how='left')
train_data = train_data.loc[:, ['mean_m', 'median_m', 'mode_m', 'std_m', 'travel_time']].values

In [55]:
test_data_df5 = df.loc[(df.time_interval_begin.dt.month == 5) & (df.time_interval_begin.dt.day >= 15) & 
                       (df.time_interval_begin.dt.hour.isin([7, 8, 14, 15, 17, 18]))
                      ]
test_history = df.loc[(df.time_interval_begin.dt.month == 4)]
test_history = test_history.groupby(
    ['link_ID', 'time_interval_begin_minutes'])['travel_time'].agg(
    [('mean_m', np.mean), ('median_m', np.median), 
     ('mode_m', mode_function), ('std_m', np.std)
    ]).reset_index()
test_data_df5 = pd.merge(test_data_df5, test_history, 
                      on=['link_ID', 'time_interval_begin_minutes'], 
                      how='left')
test_data5 = test_data_df5.loc[:, ['mean_m', 'median_m', 'mode_m', 'std_m', 'travel_time']].values

In [65]:
train_result = []
for day in range(30*132):
    for hour in range(3):
        temp_data = train_data[(day * 6 * 30) + (2 * hour * 30):(day * 6 * 30) + (2 * hour * 30) + 60]
        print(len(temp_data))
        break
    break
#         if len(temp_data)<1:
#             continue
#         train_result.append(temp_data)

60


In [60]:
len(train_result)

11091